In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 57.9 MB/s eta 0:00:00


In [ ]:
import pulp

In [ ]:
def solve_mip(objective_coeffs, inequalities, bounds, variable_types):

    prob = pulp.LpProblem("MIP_Problem", pulp.LpMaximize)

    variables = []
    for i, var_type in enumerate(variable_types):
        lb, ub = bounds[i]
        if var_type == "Continuous":
            var = pulp.LpVariable(f"x_{i}", lb, ub, cat="Continuous")
        elif var_type == "Integer":
            var = pulp.LpVariable(f"x_{i}", lb, ub, cat="Integer")
        elif var_type == "Binary":
            var = pulp.LpVariable(f"x_{i}", 0, 1, cat="Binary")
        variables.append(var)

    objective = pulp.lpSum([objective_coeffs[i] * variables[i] for i in range(len(objective_coeffs))])
    prob += objective

    for (coefficients, sense, rhs) in inequalities:
        constraint_expr = pulp.lpSum([coefficients[i] * variables[i] for i in range(len(coefficients))])
        if sense == "<=":
            prob += (constraint_expr <= rhs)
        elif sense == ">=":
            prob += (constraint_expr >= rhs)
        elif sense == "==":
            prob += (constraint_expr == rhs)

    prob.solve()

    solution_status = pulp.LpStatus[prob.status]
    print("Solution Status:", solution_status)

    # Run diagnosis
    # infeasible_constraints = diagnose_constraints(prob, prob.constraints.values())
    # print("Potential infeasible constraints:", infeasible_constraints)

    solution = {v.name: v.varValue for v in variables}

    return solution

In [ ]:
# Fixed cost of building a bus, tram, or subway line. (x1000)
fixed_costs = [8000, 8000, 20000, 20000, 45000]

# Cost of building a station for a bus, tram, or subway. (x1000)
station_cost = [100, 100, 500, 500, 2000]

# Population densities for each line. (ppl / km2)
population_densities = [1800, 1200, 400, 1100, 2000, 1300, 600, 1100, 1100, 500, 1200, 1300, 2000]

# Distance of lines in each district. (km)
distances = [40, 32, 48, 35, 45, 32, 40, 29, 35, 24, 32, 29, 48]

# Types of public transport.
public_transport_types = ['bus1', 'bus2', 'tram', 'tram_ext', 'subway']

# Cost per vehicle for each type. (x1000)
transport_costs = [250, 300, 5000, 6000, 20000]

# Capacities per each transport type.
capacities = [40, 60, 150, 250, 300]

# Speed of each vehicle. (km/h)
vehicle_speeds = [25, 25, 30, 30, 50]

# Emissions for each vehicle. (CO2 / passanger-km)
vehicle_emissions = [0.2, 0.2, 0.1, 0.1, 0.05]

# Area of each district. (km2)
district_areas = [80, 75, 120, 70, 95, 85, 110, 65, 90, 50, 75, 70, 100]

# The number of operating hours per day of each transport type.
operating_hours = [8, 8, 10, 10, 12]

# The percentage of people that prefer each type of transport.
percentages = [0.3, 0.3, 0.2, 0.2, 0.5]

# The total budget for the project. (x1000)
total_budget = 5e6

# Weights for the objective function (used to adimensionalize it).
w1 = 0.7 # Weight for the density of stations
w2 = 0.9 # Weight for the total capacity
alpha = 0.4 # Weight for the total emissions
betta = 1.2 # Weight for the total travel time

# Coefficients for the objective function.
objective_coeffs = []

objective_coeffs.extend([w1 / district_areas[i] for i in range(13) for _ in range(5)])
objective_coeffs.extend([w2 * capacities[m] / population_densities[i] for i in range(13) for m in range(5)])
objective_coeffs.extend([0] * 65)
objective_coeffs.extend([-betta * operating_hours[m] * population_densities[i] * district_areas[i] * percentages[m] * distances[i] / vehicle_speeds[m] -betta * operating_hours[m] * population_densities[i] * district_areas[i] * percentages[m] / 2 for i in range(13) for m in range(5)])
objective_coeffs.extend([-alpha] * 65)

# Constraints
constraints = []

# Cost constraint
new_coefficients = (
    [station_cost[m] for i in range(13) for m in range(5)] +
    [transport_costs[m] for i in range(13) for m in range(5)] +
    [fixed_costs[m] for i in range(13) for m in range(5)] +
    [0] * 130
)

constraints.append((new_coefficients, "<=", total_budget))

# 65 coeffs for s, 65 for v, 65 for y, 65 for f, 65 for e

# No stations of that type can be built if line not built in district
for i in range(65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i] = 1
  constraint_coefficients[i + 130] = -1e3

  constraints.append((constraint_coefficients, "<=", 0))

# No vehicles of that type can be bought in that district if line not built in district
for i in range(65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 65] = 1
  constraint_coefficients[i + 130] = -1e3

  constraints.append((constraint_coefficients, "<=", 0))

k = 0
for i in range(65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i] = 1
  if i % 5 == 0 and i != 0:
    k += 1
  constraints.append((constraint_coefficients, ">=", distances[k] / 5))

k = 0
for i in range(65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i] = 1
  if i % 5 == 0 and i != 0:
    k += 1
  constraints.append((constraint_coefficients, "<=", distances[k] / 2))

for i in range(13):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 65] = capacities[0]
  constraint_coefficients[i + 70] = capacities[1]
  constraint_coefficients[i + 75] = capacities[2]
  constraint_coefficients[i + 80] = capacities[3]
  constraint_coefficients[i + 85] = capacities[4]

  constraints.append((constraint_coefficients, ">=", population_densities[i]))

for i in range(52):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 195] = 1
  constraint_coefficients[i + 130] = -10 / 60

  constraints.append((constraint_coefficients, "<=", 0))

for i in range(52):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 195] = 1
  constraint_coefficients[i + 130] = -5 / 60

  constraints.append((constraint_coefficients, ">=", 0))

for i in range(52, 65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 195] = 1
  constraint_coefficients[i + 130] = -3 / 60

  constraints.append((constraint_coefficients, ">=", 0))

for i in range(52, 65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 195] = 1
  constraint_coefficients[i + 130] = -7 / 60

  constraints.append((constraint_coefficients, "<=", 0))


k = 0
for i in range(65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 260] = 1
  if i % 5 == 0 and i != 0:
    k += 1
  constraint_coefficients[i + 65] = -capacities[i % 5] * vehicle_emissions[i % 5] * distances[k]
  constraints.append((constraint_coefficients, "==", 0))

for i in range(0, 65, 5):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i + 130] = 1
  constraint_coefficients[i + 131] = 1
  constraint_coefficients[i + 132] = 1
  constraint_coefficients[i + 133] = 1
  constraint_coefficients[i + 134] = 1

  constraints.append((constraint_coefficients, ">=", 1))

for i in range(65):
  constraint_coefficients = [0] * 325
  constraint_coefficients[i] = -1
  constraint_coefficients[i + 65] = 1

  constraints.append((constraint_coefficients, ">=", 0))

bounds = []
variable_types = []
for i in range(130):
  bounds.append((0, None))
  variable_types.append("Integer")

for i in range(65):
  bounds.append((0, 1))
  variable_types.append("Binary")

for i in range(130):
  bounds.append((0, None))
  variable_types.append("Continuous")

In [ ]:
print("Objective Coefficients:", objective_coeffs[:10])
print("First Constraint Example:", constraints[0])
print("Variable Bounds:", bounds[:10])
print("Variable Types:", variable_types[:10])

Objective Coefficients: [0.01875, 0.01875, 0.01875, 0.01875, 0.01875, 0.02, 0.02, 0.02, 0.02, 0.02]
First Constraint Example: ([100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 100, 100, 500, 500, 2000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 250, 300, 5000, 6000, 20000, 8000, 8000, 20000, 20000, 45000, 8000, 8000, 20000, 20000, 45000, 8000, 8000, 20000, 20000, 45000, 8000, 8000, 20000, 20000, 45000, 8000, 8000, 20000, 20000,

In [ ]:
def diagnose_constraints(prob, constraints):
    infeasible_constraints = []
    for constraint in constraints:
        # Temporarily remove the constraint
        prob.constraints.pop(constraint.name)
        prob.solve()

        # Check if problem becomes feasible
        if pulp.LpStatus[prob.status] == 'Optimal':
            infeasible_constraints.append(constraint.name)
        # Re-add the constraint
        prob += constraint

    return infeasible_constraints




In [ ]:
solution = solve_mip(objective_coeffs, constraints, bounds, variable_types)

print("Optimal solution:", solution)

Solution Status: Optimal
Optimal solution: {'x_0': 8.0, 'x_1': 8.0, 'x_2': 8.0, 'x_3': 8.0, 'x_4': 8.0, 'x_5': 7.0, 'x_6': 7.0, 'x_7': 7.0, 'x_8': 7.0, 'x_9': 7.0, 'x_10': 10.0, 'x_11': 10.0, 'x_12': 10.0, 'x_13': 10.0, 'x_14': 10.0, 'x_15': 7.0, 'x_16': 7.0, 'x_17': 7.0, 'x_18': 7.0, 'x_19': 7.0, 'x_20': 9.0, 'x_21': 9.0, 'x_22': 9.0, 'x_23': 9.0, 'x_24': 9.0, 'x_25': 7.0, 'x_26': 7.0, 'x_27': 7.0, 'x_28': 7.0, 'x_29': 7.0, 'x_30': 8.0, 'x_31': 8.0, 'x_32': 8.0, 'x_33': 8.0, 'x_34': 8.0, 'x_35': 6.0, 'x_36': 6.0, 'x_37': 6.0, 'x_38': 6.0, 'x_39': 6.0, 'x_40': 7.0, 'x_41': 7.0, 'x_42': 7.0, 'x_43': 7.0, 'x_44': 7.0, 'x_45': 5.0, 'x_46': 5.0, 'x_47': 5.0, 'x_48': 5.0, 'x_49': 5.0, 'x_50': 7.0, 'x_51': 7.0, 'x_52': 7.0, 'x_53': 7.0, 'x_54': 7.0, 'x_55': 6.0, 'x_56': 6.0, 'x_57': 6.0, 'x_58': 6.0, 'x_59': 6.0, 'x_60': 10.0, 'x_61': 10.0, 'x_62': 10.0, 'x_63': 10.0, 'x_64': 10.0, 'x_65': 8.0, 'x_66': 8.0, 'x_67': 8.0, 'x_68': 8.0, 'x_69': 8.0, 'x_70': 7.0, 'x_71': 7.0, 'x_72': 7.0, 'x_73':